In [1]:
# imports
import os
import sys
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.mplot3d import Axes3D

current_dir = os.path.dirname(os.path.realpath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the system path
sys.path.insert(0, parent_dir)
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
# from pixel_setter import play_video
from EMA_functions import *
from DIC_functions import *
import glob
from sdypy import EMA
from sdypy.EMA import stabilization
import pickle as pkl
from scipy.signal import find_peaks


import matplotlib.pyplot as plt
import numpy as np

# Import test data
df_file_description = pd.read_csv('H:/My Drive/PHD/HSC/file_descriptions_wEMA.csv')
# Back up the data
df_file_description.to_csv('H:/My Drive/PHD/HSC/file_descriptions_wEMA_backup.csv')

# List all files
files_cam = glob.glob('D:/thijsmas/HSC/**/*_cam.pkl', recursive=True)
files_ema = glob.glob('D:/thijsmas/HSC/**/*_EMA.pkl', recursive=True)
files_cam_d = glob.glob('D:/thijsmas/HSC/**/*_cam_damped.pkl', recursive=True)
files_ema_d = glob.glob('D:/thijsmas/HSC/**/*_EMA_damped.pkl', recursive=True)

df_file_description

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


,Unnamed: 0,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,...,upper,pol_order_high,added_damping,near_zero_threshold,approximate_height,peak_F,peak_F_threshold,shift,test_number,higher
0,0,Full_web_ecc1_new_Floc1_nomass_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc1\web0\no_mass\Full_web...,1,1,False,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.7854421530864197,0.5,"(0, 0)",1.0,300.0
1,1,Full_web_ecc1_new_Floc8_nomass_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc1\web0\no_mass\Full_web...,1,8,False,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.7547670153086419,0.5,"(0, 0)",1.0,300.0
2,2,Full_web_ecc1_new_Floc2_nomass_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc1\web0\no_mass\Full_web...,1,2,False,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.6382718360493826,0.5,"(0, 0)",1.0,300.0
3,3,Full_web_ecc1_new_Floc9_nomass_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc1\web0\no_mass\Full_web...,1,9,False,Loose,Impact,0,10100,...,NaN,150.0,NaN,NaN,NaN,2.090359554567901,0.5,"(0, 0)",1.0,300.0
4,4,Full_web_ecc1_new_Floc9_nomass_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc1\web0\no_mass\Full_web...,1,9,False,Loose,Impact,1,10000,...,NaN,150.0,NaN,NaN,NaN,1.4239227160493826,0.5,"(0, 0)",1.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,214,Full_web_ecc0_new2_Floc18_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web1\Full_web_ecc0_ne...,0,18,True,Loose,Impact,1,8000,...,NaN,150.0,NaN,NaN,NaN,4.122028446419753,0.5,"(0, 0)",1.0,300.0
215,215,Full_web_ecc0_new2_Floc19_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web1\Full_web_ecc0_ne...,0,19,True,Loose,Impact,0,8000,...,NaN,150.0,NaN,NaN,NaN,3.134645756049382,0.5,"(0, 0)",1.0,300.0
216,216,Full_web_ecc0_new2_Floc19_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web1\Full_web_ecc0_ne...,0,19,True,Loose,Impact,1,8000,...,NaN,150.0,NaN,NaN,NaN,4.090599619753086,0.5,"(0, 0)",1.0,300.0
217,217,Full_web_ecc0_new2_Floc5_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web1\Full_web_ecc0_ne...,0,5,True,Loose,Impact,0,8000,...,NaN,150.0,NaN,NaN,NaN,1.6640479960493826,0.5,"(0, 0)",1.0,300.0


In [5]:
ecc = 0
filtered_df = df_file_description.query(f"ecc == {ecc} & mass == True & `taut/loose` == 'Loose' & `impact/pluck` == 'Impact'") # & version == 1
valid_plots = []
for index, row, in filtered_df.iterrows():
    # Access individual columns using row['column_name']
    file_name = row['filename']
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])

    base_filename = os.path.splitext(file_name)[0]
    # Find the corresponding cam file
    cam_file = None
    for file_path in files_cam:
        if base_filename in file_path:
            cam_file = file_path
            break
    if cam_file:
        valid_plots.append(True)
    else:
        valid_plots.append(False)
filtered_df = filtered_df[valid_plots]
filtered_df


,Unnamed: 0,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,...,upper,pol_order_high,added_damping,near_zero_threshold,approximate_height,peak_F,peak_F_threshold,shift,test_number,higher
172,172,Full_web_ecc0_new_Floc1_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,1,True,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.2938108908641974,0.5,"(0, 0)",1.0,300.0
173,173,Full_web_ecc0_new_Floc1_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,1,True,Loose,Impact,1,10000,...,NaN,150.0,NaN,NaN,NaN,1.6161385056790123,0.5,"(0, 0)",1.0,300.0
174,174,Full_web_ecc0_new_Floc2_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,2,True,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.710525477530864,0.5,"(0, 0)",1.0,300.0
175,175,Full_web_ecc0_new_Floc2_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,2,True,Loose,Impact,1,10000,...,NaN,150.0,NaN,NaN,NaN,1.42568132345679,0.5,"(0, 0)",1.0,300.0
176,176,Full_web_ecc0_new_Floc2_v2_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,2,True,Loose,Impact,2,10000,...,NaN,150.0,NaN,NaN,NaN,2.711826289382716,0.5,"(0, 0)",1.0,300.0
177,177,Full_web_ecc0_new_Floc2_v3_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,2,True,Loose,Impact,3,10000,...,NaN,150.0,NaN,NaN,NaN,3.2987238271604933,0.5,"(0, 0)",1.0,300.0
178,178,Full_web_ecc0_new_Floc3_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,3,True,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,1.7480089382716049,0.5,"(0, 0)",2.0,300.0
179,179,Full_web_ecc0_new_Floc3_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,3,True,Loose,Impact,1,10000,...,NaN,150.0,NaN,NaN,NaN,0.5239930597530864,0.5,"(0, 0)",1.0,300.0
180,180,Full_web_ecc0_new_Floc3_v2_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,3,True,Loose,Impact,2,10000,...,NaN,150.0,NaN,NaN,NaN,2.4223595101234565,0.5,"(0, 0)",1.0,300.0
181,181,Full_web_ecc0_new_Floc4_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,4,True,Loose,Impact,0,10000,...,NaN,150.0,NaN,NaN,NaN,2.426253569382716,0.5,"(0, 0)",1.0,300.0


In [6]:
indices = df_file_description.index[df_file_description.index.isin(filtered_df.index)]
if ecc == 0:
    new_spider_ij_value = '(528, 258)' # ecc 0
elif ecc == 1:
    new_spider_ij_value = '(533, 223)'      # ecc 1
elif ecc == 2:
    pass

if ecc == 0 or ecc == 1:
    for idx, row in filtered_df.iterrows():
        df_file_description.loc[idx, 'spider_ij'] = new_spider_ij_value
        filtered_df.loc[idx, 'spider_ij'] = new_spider_ij_value


Ecc 2, version 1

In [7]:
%matplotlib qt
fig, ax = plt.subplots(5, 4, figsize=(15, 10))
fig3, ax3 = plt.subplots(1, 1, figsize=(15, 10))
first_row = filtered_df.iloc[3]
file_name = first_row['filename']
file_path = first_row['path']
spider_ij_0 = eval(first_row['spider_ij'])
prey_ij = eval(first_row['prey_ij'])

# Load the video and plot the first frame
file_path = first_row['path']
DIC_structure = DIC_Structure(file_path)
video = DIC_structure.video
ax3.imshow(video.mraw[0], cmap='gray')
ax3.plot(spider_ij_0[0], spider_ij_0[1], 'go')
ax3.plot(prey_ij[0], prey_ij[1], 'g*')
ax3.axis('off')
margin_x = 400
margin_y = 400

height_im, width_im = video.mraw[0].shape

angle_vec = []
for (index, row,), ax_i in zip(filtered_df.iterrows(), ax.flatten()):
    file_name = row['filename']
    file_path = row['path']
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])

    prey_ij_rel0 = (prey_ij[0] - spider_ij_0[0] + spider_ij[0], prey_ij[1] - spider_ij_0[1]+ spider_ij[1])
    prey_ij_rel = (prey_ij[0] - spider_ij_0[0] + spider_ij[0] - width_im/2, prey_ij[1] - spider_ij_0[1]+ spider_ij[1]- height_im/2)
    angle = np.arctan2(prey_ij_rel[1], prey_ij_rel[0])
    angle_vec.append(angle)

    DIC_structure = DIC_Structure(file_path)
    video = DIC_structure.video

    ax_i.set_title(file_name)
    ax_i.imshow(video.mraw[0], cmap='gray')
    # ax_i.plot(spider_ij_0[0], spider_ij_0[1], 'bo')
    print(spider_ij)
    ax_i.plot(spider_ij[0], spider_ij[1], 'go')
    # ax_i.plot(prey_ij_rel0[0], prey_ij_rel0[1], 'b*')
    ax_i.plot(prey_ij[0], prey_ij[1], 'g*')


angle_vec = np.mod(angle_vec, 2*np.pi)
order = np.argsort(angle_vec)

n_valid = len(filtered_df)
# Sort valid angles and get the sorted indices

width = 0.2
height = 0.15
R_vec = np.tile([0.55, 0.35], n_valid // 2 + 1)[:n_valid]
x0 = 0.4
y0 = 0.4

angles_even_distributed = np.linspace(0, -2*np.pi, n_valid, endpoint=False)
# angles_even_distributed = angles_even_distributed[order]
# Calculate inset positions using vectorized operations
inset_positions = np.column_stack((
    x0 + R_vec * np.cos(angles_even_distributed),
    y0 + R_vec * np.sin(angles_even_distributed),
    np.full(n_valid, width),
    np.full(n_valid, height)
))

inset_axs = [
    inset_axes(ax3, width="100%", height="100%",
               bbox_to_anchor=pos, bbox_transform=ax3.transAxes, loc='center')
    for pos in inset_positions
]

for id2, idx in enumerate(order):
    row = filtered_df.iloc[idx]
    file_name = row['filename']
    loc = row['loc']
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])
    peak_F = eval(row['peak_F'])
    pos = inset_positions[id2]
    inset_ax = inset_axs[id2]

    base_filename = os.path.splitext(file_name)[0]
    # Find the corresponding cam file
    cam_file = None
    for file_path in files_cam:
        if base_filename in file_path:
            cam_file = file_path
            break
    with open(cam_file, 'rb') as f:
        cam = pkl.load(f)
    
    ema_file = None
    for file_path in files_ema:
        if base_filename in file_path:
            ema_file = file_path
            break
    with open(ema_file, 'rb') as f:
        ema = pkl.load(f)
    print(ema_file)

    ax3.plot(spider_ij_0[0], spider_ij_0[1], 'bo')
    prey_ij_rel = (prey_ij[0] - spider_ij_0[0] + spider_ij[0], prey_ij[1] - spider_ij_0[1]+spider_ij[1])
    ax3.text(prey_ij_rel[0], prey_ij_rel[1], id2+1, color='blue')

    ax3.plot(prey_ij_rel[0], prey_ij_rel[1], 'go')
    ################For FRF:
    # frf = np.abs(np.mean(cam.frf, axis=0))
    # frf = frf / np.max(frf)

    # for inset_ax_temp in inset_axs:
    #     inset_ax_temp.semilogy(cam.freq, frf, 'k-', label=loc, lw=0.5, alpha=0.5)
    # inset_ax.semilogy(cam.freq, frf, 'r-', label=loc, zorder=10)
    
    # try:
    #     for freq in np.array(cam.nat_freq[:10]):
    #         inset_ax.axvline(freq, color='r', linestyle='--', lw=0.5)
    # except:
    #     pass

    # inset_ax.set_xlim([0, 50])
    # inset_ax.set_ylim([1e-3, 2])
    ###############

    ############################## For PSD:
    S_xx = 1/ema.t_camera[-1] * np.conj(ema.disp_fft_y[ema.valid_tps]) * ema.disp_fft_y[ema.valid_tps]
    S_xx = np.mean(S_xx, axis=0)#/peak_F
    arg_fn1 = np.argmin(np.abs(ema.freq_camera - 7.4))
    S_xx_peaks, _ = find_peaks(S_xx)

    S_xx = S_xx / np.abs(S_xx[S_xx_peaks[0]])
    for inset_ax_temp in inset_axs:
        inset_ax_temp.semilogy(ema.freq_camera, S_xx, 'k-', label=loc, lw=0.5, alpha=0.5)
    inset_ax.semilogy(ema.freq_camera, S_xx, 'b-', label=loc, zorder=10)
    inset_ax.set_xlim([0, 50])
    inset_ax.set_ylim([1e-4, 1e1])
    ###############################
    inset_ax.title.set_text(f'{id2+1}')

xlim  = ax3.set_xlim([-margin_x, width_im + margin_x])
ylim = ax3.set_ylim([height_im + margin_y, -margin_y])

for id2, idx in enumerate(order):
    row = filtered_df.iloc[idx]
    file_name = row['filename']
    loc = row['loc']
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])
    pos = inset_positions[id2]

    prey_ij_rel = (prey_ij[0] - spider_ij_0[0] + spider_ij[0], prey_ij[1] - spider_ij_0[1] + spider_ij[1])
    cam_file = None
    for file_path in files_cam:
        if base_filename in file_path:
            cam_file = file_path
            break
    center = (pos[0] + 0.5 * pos[2], pos[1] + 0.5 * pos[3])
    inset_center_x_data = center[0] * (xlim[1] - xlim[0]) + xlim[0]  # X coordinate in data
    inset_center_y_data = center[1] * (ylim[1] - ylim[0]) + ylim[0]  # Y coordinate in data
    ax3.plot([prey_ij_rel[0], inset_center_x_data], [prey_ij_rel[1], inset_center_y_data], 'r-')

xlim  = ax3.set_xlim([-margin_x, width_im + margin_x])
ylim = ax3.set_ylim([height_im + margin_y, -margin_y])




c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyidi\pyidi.py:275: DeprecationWarning: `self.mraw` is deprecated and will be removed in the next version. Please use `self.reader.mraw` instead.
  warnings.warn('`self.mraw` is deprecated and will be removed in the next version. Please use `self.reader.mraw` instead.', DeprecationWarning)


Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
(528, 258)
Ava

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\scipy\signal\_peak_finding.py:265: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\thijsmas\AppData\Roaming\Python\Python311\site-packages\matplotlib\cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
C:\Users\thijsmas\AppData\Roaming\Python\Python311\site-packages\matplotlib\cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc5_v0_S01\Full_web_ecc0_new_Floc5_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v2_S01\Full_web_ecc0_new_Floc4_v2_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v1_S01\Full_web_ecc0_new_Floc4_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc4_v0_S01\Full_web_ecc0_new_Floc4_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc7_v1_S01\Full_web_ecc0_new_Floc7_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc7_v0_S01\Full_web_ecc0_new_Floc7_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc6_v0_S01\Full_web_ecc0_new_Floc6_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc6_v1_S01\Full_web_ecc0_new_Floc6_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc6_v2_S01\Full_web_ecc0_new_Floc6_v2_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc6_v3_S01\Full_web_ecc0_new_Floc6_v3_S01.cih

Shift of S_xx due to z-location of nut keeping angle to hub approximately constant

In [8]:
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
fig2, ax2 = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': '3d'})
fig3, ax3 = plt.subplots(1, 1, figsize=(15, 10))
first_row = filtered_df.iloc[3]
file_name = first_row['filename']
file_path = first_row['path']
spider_ij_0 = eval(first_row['spider_ij'])

# Load the video and plot the first frame
file_path = first_row['path']
DIC_structure = DIC_Structure(file_path)
video = DIC_structure.video
ax.imshow(video.mraw[0], cmap='gray')
margin_x = 400
margin_y = 400

height_im, width_im = video.mraw[0].shape

include = []
n_peaks = 3
peaks_freq = []
peaks_amp = []
prey_z = []
for index, row, in filtered_df.iterrows():
    file_name = row['filename']
    base_filename = os.path.splitext(file_name)[0]
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])
    prey_ij_rel0 = (prey_ij[0] - spider_ij_0[0] + spider_ij[0], prey_ij[1] - spider_ij_0[1] + spider_ij[1])
    prey_ij_rel = (prey_ij[0] - spider_ij_0[0] + spider_ij[0] - width_im/2, prey_ij[1] - spider_ij_0[1]+ spider_ij[1]- height_im/2)
    if not np.abs(prey_ij_rel[0])<30:
        include.append(False)
        ax.plot(prey_ij_rel0[0], prey_ij_rel0[1], 'r*')
        continue
    
    include.append(True)
    ax.plot(prey_ij_rel0[0], prey_ij_rel0[1], 'go')

    ema_file = None
    for file_path in files_ema:
        if base_filename in file_path:
            ema_file = file_path
            break
    with open(ema_file, 'rb') as f:
        ema = pkl.load(f)
    print(ema_file)


    ############################## For PSD:
    S_xx = np.abs(1/ema.t_camera[-1] * np.conj(ema.disp_fft_y[ema.valid_tps]) * ema.disp_fft_y[ema.valid_tps])
    S_xx = np.mean(S_xx, axis=0)#/peak_F

    arg_fn1 = np.argmin(np.abs(ema.freq_camera - 7.4))
    S_xx_peaks_norm, _ = find_peaks(S_xx[2:arg_fn1+5])
    S_xx_peaks_norm = S_xx_peaks_norm[0] + 2
    S_xx = S_xx / np.abs(S_xx[S_xx_peaks_norm])

    log_S_xx = np.log(S_xx)
    S_xx_peaks, _ = find_peaks(S_xx, distance=5)
    peaks_freq.append(ema.freq_camera[S_xx_peaks[:n_peaks]])
    peaks_amp.append(log_S_xx[S_xx_peaks[:n_peaks]])

    y_vec = np.ones_like(S_xx) * -prey_ij_rel[1]
    prey_z.append(-prey_ij_rel[1])

    xlim_high = 50
    arg_high = np.argmin(np.abs(ema.freq_camera - xlim_high))
    ax2.plot3D(ema.freq_camera[:arg_high], y_vec[:arg_high], log_S_xx[:arg_high])
    # ax2.semilogy(ema.freq_camera, S_xx, z_vec, 'k-', label=loc, zorder=10)
    ax2.set_xlim3d([0, 50])
    ax2.set_zlim3d([-10, 2])
    # ax2.set_zscale('log')
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Z [px]')
    ax2.set_zlabel('Log10(PSD) (normalized)')
    ###############################
    ax3.plot(ema.freq_camera, log_S_xx, 'b-')
    ax3.set_xlim([0, 50])
    # ax3.set_ylim([1e-4, 1e1])
    ax3.set_ylim([-10, 2])
prey_z = np.array(prey_z)
order = np.argsort(prey_z)
peaks_freq = np.array(peaks_freq)[order]
peaks_amp = np.array(peaks_amp)[order]
for mode in range(n_peaks):
    ax2.plot3D(peaks_freq[:,mode], prey_z[order], peaks_amp[:, mode], 'k--')
include

Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc11_v0_S01\Full_web_ecc0_new2_Floc11_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc11_v1_S01\Full_web_ecc0_new2_Floc11_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc12_v0_S01\Full_web_ecc0_new2_Floc12_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc12_v1_S01\Full_web_ecc0_new2_Floc12_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc13_v0_S01\Full_web_ecc0_new2_Floc13_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc13_v1_S01\Full_web_ecc0_new2_Floc13_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc17_v0_S01\Full_web_ecc0_new2_Floc17_v0_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc17_v1_S01\Full_web_ecc0_new2_Floc17_v1_S01.cihx_EMA.pkl
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False]

Shift in S_xx due to angle of nut location with approximate constant distance to hub

In [ ]:
from pyidi import ROISelect
from matplotlib.path import Path
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
fig2, ax2 = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': '3d'})
fig3, ax3 = plt.subplots(1, 1, figsize=(15, 10))
fig4, ax4 = plt.subplots(1, 1, figsize=(15, 10))
ax4.invert_yaxis()
first_row = filtered_df.iloc[3]
file_name = first_row['filename']
file_path = first_row['path']
spider_ij_0 = eval(first_row['spider_ij'])

# Load the video and plot the first frame
file_path = first_row['path']
DIC_structure = DIC_Structure(file_path)
video = DIC_structure.video
ax.imshow(video.mraw[0], cmap='gray')

# roi_select = ROISelect(video)
# roi_select.polygon
if ecc == 0:
    polygon_new = np.array([[231, 195,-13,-13, -216, -214, 6  , 114  , -81, 7, 231],
                            [-6, 106, 125, 90, 74  , -97  , -124, -69, -78, 42, -6]]) # ecc 0
elif ecc == 1:
    polygon_new = np.array([[20, -219, -214, 6  , 114  , -81, 7, 20],
                            [79, 82  , -97  , -124, -69, -78, 42, 79]]) # ecc 1
elif ecc == 2:
    polygon_new = np.array([[-8, -219, -214, 6  , 9  , -81, 7, -8],
                            [79, 82  , -97  , -124, -98, -78, 42, 79]]) # ecc 2
ax4.plot(polygon_new[0], polygon_new[1], 'r-')

path = Path(polygon_new.T)

margin_x = 400
margin_y = 400

height_im, width_im = video.mraw[0].shape

include = []
n_peaks = 3
peaks_freq = []
peaks_amp = []
prey_theta = []
for index, row, in filtered_df.iterrows():
    file_name = row['filename']
    base_filename = os.path.splitext(file_name)[0]
    prey_ij = eval(row['prey_ij'])
    spider_ij = eval(row['spider_ij'])
    prey_ij_rel0 = (prey_ij[0] - spider_ij_0[0] + spider_ij[0], prey_ij[1] - spider_ij_0[1]+ spider_ij[1])
    prey_ij_rel = (prey_ij[0] - spider_ij_0[0] + spider_ij[0] - width_im/2, prey_ij[1] - spider_ij_0[1]+ spider_ij[1]- height_im/2)
    if not path.contains_point(prey_ij_rel):
        include.append(False)
        ax.plot(prey_ij_rel0[0], prey_ij_rel0[1], 'r*')
        ax4.plot(prey_ij_rel[0], prey_ij_rel[1], 'r*')
        continue
    
    include.append(True)
    ax.plot(prey_ij_rel0[0], prey_ij_rel0[1], 'go')
    ax4.plot(prey_ij_rel[0], prey_ij_rel[1], 'go')
    # continue
    ema_file = None
    for file_path in files_ema:
        if base_filename in file_path:
            ema_file = file_path
            break
    with open(ema_file, 'rb') as f:
        ema = pkl.load(f)
    print(ema_file)


    ############################## For PSD:
    S_xx = np.abs(1/ema.t_camera[-1] * np.conj(ema.disp_fft_y[ema.valid_tps]) * ema.disp_fft_y[ema.valid_tps])
    S_xx = np.mean(S_xx, axis=0)#/peak_F

    arg_fn1 = np.argmin(np.abs(ema.freq_camera - 7.4))
    S_xx_peaks_norm, _ = find_peaks(S_xx[2:arg_fn1+2])
    S_xx_peaks_norm = S_xx_peaks_norm[0] + 2
    S_xx = S_xx / S_xx[S_xx_peaks_norm]

    log_S_xx = np.log(S_xx)
    S_xx_peaks, _ = find_peaks(S_xx, distance=5)
    peaks_freq.append(ema.freq_camera[S_xx_peaks[:n_peaks]])
    peaks_amp.append(log_S_xx[S_xx_peaks[:n_peaks]])

    angle_temp = np.arctan2(prey_ij_rel[0], -prey_ij_rel[1]) + np.pi
    y_vec = np.ones_like(S_xx) * angle_temp
    prey_theta.append(angle_temp)

    xlim_high = 50
    arg_high = np.argmin(np.abs(ema.freq_camera - xlim_high))
    ax2.plot3D(ema.freq_camera[:arg_high], y_vec[:arg_high], log_S_xx[:arg_high])
    # ax2.semilogy(ema.freq_camera, S_xx, z_vec, 'k-', label=loc, zorder=10)
    ax2.set_xlim3d([0, 50])
    ax2.set_zlim3d([-10, 2])
    # ax2.set_zscale('log')
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Angle [rad]')
    ax2.set_zlabel('Log10(PSD) (normalized)')
    ###############################
    ax3.plot(ema.freq_camera, log_S_xx, 'b-')
    ax3.set_xlim([0, 50])
    # ax3.set_ylim([1e-4, 1e1])
    ax3.set_ylim([-10, 2])
prey_theta = np.array(prey_theta)
order = np.argsort(prey_theta)
peaks_freq = np.array(peaks_freq)[order]
peaks_amp = np.array(peaks_amp)[order]
for mode in range(n_peaks):
    ax2.plot3D(peaks_freq[:,mode], prey_theta[order], peaks_amp[:, mode], 'k--')
include